In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)

In [3]:
# Используем системную команду head для просмотра файла на HDFS
!hdfs dfs -cat data/2022-11-04.txt | head -20

# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario
1832792610,2022-11-04 14:22:18,0,53,63.58,101139738,1170,0,0
1832792611,2022-11-04 02:12:24,0,53,92.95,101095944,1170,0,0
1832792612,2022-11-04 12:49:35,3,205,48.88,101134175,1170,0,0
1832792613,2022-11-04 02:40:01,5,383,24.69,101097601,1170,0,0
1832792614,2022-11-04 08:02:05,6,858,95.48,101116925,1170,0,0
1832792615,2022-11-04 05:45:04,8,931,60.98,101108704,1170,0,0
1832792616,2022-11-04 20:01:50,8,931,28.48,101160110,1170,0,0
1832792617,2022-11-04 15:11:42,9,450,7.89,101142702,1170,0,0
1832792618,2022-11-04 11:20:49,10,549,63.37,101128849,1170,0,0
1832792619,2022-11-04 23:11:46,10,549,78.02,101171506,1170,0,0
1832792620,2022-11-04 10:26:06,11,337,68.54,101125566,1170,0,0
1832792621,2022-11-04 14:13:55,11,337,12.04,101139235,1170,0,0
1832792622,2022-11-04 04:17:01,11,975,76.00,101103421,1170,0,0
1832792623,2022-11-04 07:39:18,11,975,12.08,101115558,1170

In [ ]:
# Файл имеет заголовок с разделителем '|', но данные с разделителем ','
# Опеределяем схему
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, IntegerType, TimestampType
from pyspark.sql.functions import col, to_timestamp

# Вариант 1: Читаем datetime как строку, потом конвертируем
schema = StructType([
    StructField("tranaction_id", LongType(), True),
    StructField("tx_datetime", StringType(), True),  # сначала как строка
    StructField("customer_id", IntegerType(), True),
    StructField("terminal_id", IntegerType(), True),
    StructField("tx_amount", DoubleType(), True),
    StructField("tx_time_seconds", LongType(), True),
    StructField("tx_time_days", IntegerType(), True),
    StructField("tx_fraud", IntegerType(), True),
    StructField("tx_fraud_scenario", IntegerType(), True)
])

# Читаем файл и конвертируем datetime в правильный тип
df = (
    spark.read.csv(
        "data/2022-11-04.txt",
        sep=",",           # разделитель - запятая
        schema=schema      # используем определённую схему
    )
    .filter(col("tranaction_id").isNotNull())  # убираем строку с заголовком
    .withColumn("tx_datetime", to_timestamp("tx_datetime", "yyyy-MM-dd HH:mm:ss"))  # конвертируем в timestamp
)

Проверим схему данных с правильным типом datetime

In [1]:
# Проверим типы данных
df.printSchema()

# Посмотрим на данные
df.show(5, truncate=False)

NameError: name 'df' is not defined

### Преимущества использования TimestampType:
1. **Меньше места** в parquet (эффективное хранение)
2. **Быстрые операции** с датами (фильтрация, сортировка, группировка)
3. **Встроенные функции** для работы с датами (year, month, day, hour и т.д.)
4. **Автоматическая валидация** формата даты

In [ ]:
# Пример работы с datetime колонкой
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, dayofweek

# Извлечём различные компоненты даты
df.select(
    "tx_datetime",
    year("tx_datetime").alias("year"),
    month("tx_datetime").alias("month"),
    dayofmonth("tx_datetime").alias("day"),
    hour("tx_datetime").alias("hour"),
    minute("tx_datetime").alias("minute"),
    dayofweek("tx_datetime").alias("day_of_week")  # 1 = Воскресенье, 7 = Суббота
).show(5, truncate=False)

In [ ]:
# Сохраним в parquet
(
    riiid_df
        .write
        .mode("overwrite")
        .parquet("data/2022-11-04.parquet")
)

In [ ]:
# Посмотрим содержимое директории parquet
!hdfs dfs -ls data/2022-11-04.parquet

print("\n" + "="*50)
print("Размер файлов внутри parquet директории:")
print("="*50)
!hdfs dfs -du -h data/2022-11-04.parquet

In [4]:
# Читаем parquet обратно
df_from_parquet = spark.read.parquet("data/2022-11-04.parquet")

print(f"Количество записей в parquet: {df_from_parquet.count()}")
print(f"Схема данных:")
df_from_parquet.printSchema()
print("\nПервые 5 записей:")
df_from_parquet.show(5)

Количество записей в parquet: 46998983
Схема данных:
root
 |-- tranaction_id: long (nullable = true)
 |-- tx_datetime: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- terminal_id: integer (nullable = true)
 |-- tx_amount: double (nullable = true)
 |-- tx_time_seconds: long (nullable = true)
 |-- tx_time_days: integer (nullable = true)
 |-- tx_fraud: integer (nullable = true)
 |-- tx_fraud_scenario: integer (nullable = true)


Первые 5 записей:
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|   1838826044|2022-11-07 15:08:53|     850577|        324|    65.72|      101401733|        1173|       0|                0|
|   1838826045|2022-11